-----
# cartoGRAPHs - FIGURE 2B Suppl.
precalculated large networks , e.g. the Interactome

A Notebook to produce 2D and 3D network layouts from any Graph,
including interactive visualization (html files) and export functions 
to import into the VRNetzer analytics platform by Pirch et al.

-----

In [1]:
from cartoGRAPHs import * 

from func_load_data import *
from func_visual_properties import * 
from func_calculations import * 
from func_embed_plot import *
from func_exportVR import * 

2021-10-13 18:22:12.079049: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-13 18:22:12.079440: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


_____
# HUMAN NETWORK
_____

In [2]:
organism = 'human'

G = load_graph(organism) 

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

In [3]:
d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

_____ 
# LOAD DATA
_____

In [5]:
# get Layout basis --> global layout matrix 
DF_structural = load_datamatrix(G,organism, 'global')
df_max = DF_structural.max()
l_max_visprob = max(list(df_max.values))


# get Disease Feature Matrix
FM_disease = pd.read_csv('input/Features_Disease_Dataframe_'+organism+'.csv', index_col=0)

# VECTOR : NF1 GENE + DISEASE MODULE (8712 - neurofibromatosis)
e.g.
+ DOID:8712 / Neurofibromatosis
+ DOID:962 / Neurofibroma
+ DOID:0050736 / Autosomal Dominant Disease

In [178]:
path_nf = 'input/neurofibromatosis/'

# gene of interest
nf_goi_ = str(int(np.loadtxt(path_nf+'nf_gene_of_interest.txt', unpack=False)))
nf_goi = list(nf_goi_.split(" "))
nf_goi_str = ' '.join([str(item) for item in nf_goi])

nf_neighbors = []
for edge in G.edges():
    if edge[0] == nf_goi_str:
        nf_neighbors.append(edge[1])
    elif edge[1] == nf_goi_str: 
        nf_neighbors.append(edge[0])

nf_all = nf_neighbors + nf_goi

# get disease module genes 
num_doid = '8712' 

# define specific DOID 
dismod = FM_disease.loc[:,'DOID:'+num_doid]
dismod.index = FM_disease.index
dismod_id = 'DOID'+num_doid

dismod_genes = [] 
for i,v in enumerate(dismod.values):
    if v == 1:
        dismod_genes.append(dismod.index[i])
    else:
        pass

print('Number of Genes associated to DOID', num_doid,':',len(dismod_genes))

l_genes_dismod = [str(i) for i in dismod_genes]
l_genes_dismod_with = [str(i) for i in dismod_genes] + nf_neighbors
#l_genes_dismod = l_genes_dismod_with

print('Genes to be enhanced:', len(l_genes_dismod))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod = pd.DataFrame(dismod.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod.index:
    if i in l_genes_dismod:
        DF_dismod.loc[i] = 1
    
DF_dismod.columns = ['Neurofib 8712']

Number of Genes associated to DOID 8712 : 12
Genes to be enhanced: 12


#### SPECIFIC NODE + EDGE COLORS

In [179]:
##########################

# colors for DOID 8712 

##########################

node_col_dismod = '#00D9FC' #'#008792' #'#00E3DF' #'#FF8E00' 

d_edge_col_dismod= color_edges_from_nodelist_specific(G, l_genes_dismod, node_col_dismod)
print('Links of DOID 8712 genes:',len(d_edge_col_dismod))

d_edge_col_dismod_with = color_edges_from_nodelist_specific(G, l_genes_dismod_with, node_col_dismod)
print('Links of DOID 8712 genes:',len(d_edge_col_dismod_with))

Links of DOID 8712 genes: 3
Links of DOID 8712 genes: 140


# VECTORS : Disease Context 
+ DOID:3165 - Skin benign neoplasm
+ DOID:10534 - Stomach Cancer
+ DOID:3620 - Central Nervous System Cancer
+ DOID:0060089 - Endocrine organ benign neoplasm
+ resources: https://disease-ontology.org/ https://en.wikipedia.org/wiki/Neurofibromatosis

### First Disease Vector | DOID:3165 - Skin benign neoplasm

In [283]:
# -----------------------------
num_doid_1 = '3165'
# -----------------------------


# define specific DOID 
dismod_1 = FM_disease.loc[:,'DOID:'+num_doid_1]
dismod_1.index = FM_disease.index

# get all genes associated to disease
dismod_genes_1 = [] 
for i,v in enumerate(dismod_1.values):
    if v == 1:
        dismod_genes_1.append(dismod_1.index[i])
    else:
        pass

l_genes_dismod_1 = [str(i) for i in dismod_genes_1]
#print('Number of Genes associated to DOID', num_doid_1,':',len(l_genes_dismod_1))

overlap_1 = []
for i in dismod_genes_1:
    if str(i) in nf_all or int(i) in nf_all:
    #if str(i) in dismod_genes or int(i) in dismod_genes:
        overlap_1.append(str(i))
#print('Number of Genes overlap of NF1 neighbors and Disease Module:', (len(overlap_1)))

l_genes_dismod_1_with = l_genes_dismod_1 + overlap_1 # + nf_neighbors[:10]
#print('Number of Genes associated to DOID', num_doid_1,':',len(l_genes_dismod_1_with))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_1 = pd.DataFrame(dismod_1.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_1.index:
    if i in l_genes_dismod_1: #_with:
        DF_dismod_1.loc[i] = 1

#### SPECIFIC NODE + EDGE COLORS

In [282]:
# color nodes
node_col_1 = '#FF7001' # (orange) #'#B00160' 

# color edges    
d_edge_col_1 = color_edges_from_nodelist_specific(G, l_genes_dismod_1_with, 
                                                  node_col_1)
len(d_edge_col_1)

446

### Second Disease Vector | DOID:10534 - Stomach Cancer

In [279]:
# -----------------------------
num_doid_2 = '10534' #'3883' #'1612' #'3490'
# -----------------------------

# define specific DOID 
dismod_2 = FM_disease.loc[:,'DOID:'+num_doid_2]
dismod_2.index = FM_disease.index

# get all genes associated to disease
dismod_genes_2 = [] 
for i,v in enumerate(dismod_2.values):
    if v == 1:
        dismod_genes_2.append(dismod_2.index[i])
    else:
        pass
    
l_genes_dismod_2 = [str(i) for i in dismod_genes_2]
#print('Number of Genes associated to DOID', num_doid_2,':',len(l_genes_dismod_2)) 

overlap_2 = []
for i in dismod_genes_2:
    if str(i) in nf_all or int(i) in nf_all:
    #if str(i) in dismod_genes or int(i) in dismod_genes:
        overlap_2.append(str(i))
#print('Number of Genes overlap of NF1 neighbors and Disease Module:', (len(overlap_2)))
#print(overlap_2) 

l_genes_dismod_2_with = l_genes_dismod_2 + overlap_2 # + nf_neighbors[15:25]
#print('Number of Genes associated to DOID', num_doid_2,':',len(l_genes_dismod_2_with))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_2 = pd.DataFrame(dismod_2.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_2.index:
    if i in l_genes_dismod_2_with:
        DF_dismod_2.loc[i] = 1

#### SPECIFIC NODE + EDGE COLORS

In [280]:
# color nodes
node_col_2 = '#FFC433' #(yellow)

# color edges 
d_edge_col_2 = color_edges_from_nodelist_specific(G, l_genes_dismod_2_with, node_col_2)
len(d_edge_col_2)

910

### Third Disease Vector | DOID:3620 - Central Nervous System Cancer

In [285]:
# -----------------------------
num_doid_3 = '3620' #'0060115' 
# -----------------------------

# define specific DOID 
dismod_3 = FM_disease.loc[:,'DOID:'+num_doid_3]
dismod_3.index = FM_disease.index

dismod_genes_3 = [] 
for i,v in enumerate(dismod_3.values):
    if v == 1:
        dismod_genes_3.append(dismod_3.index[i])
    else:
        pass

l_genes_dismod_3 = [str(i) for i in dismod_genes_3]
#print('Number of Genes associated to DOID', num_doid_3,':',len(l_genes_dismod_3))

overlap_3 = []
for i in dismod_genes_3:
    if str(i) in nf_all or int(i) in nf_all:
    #if str(i) in dismod_genes or int(i) in dismod_genes:
        overlap_3.append(str(i))
#print('Number of Genes overlap of NF1 neighbors and Disease Module:', (len(overlap_3)))

l_genes_dismod_3_with = l_genes_dismod_3 + overlap_3 # + nf_neighbors[30:40]
#print('Number of Genes associated to DOID', num_doid_3,':',len(l_genes_dismod_3_with))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_3 = pd.DataFrame(dismod_3.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_3.index:
    if i in l_genes_dismod_3: #_with:
        DF_dismod_3.loc[i] = 1

#### SPECIFIC NODE + EDGE COLORS

In [286]:
# color nodes
node_col_3 = '#BEDC00' # (yellowgreen)

# color edges    
d_edge_col_3 = color_edges_from_nodelist_specific(G, l_genes_dismod_3_with, 
                                                  node_col_3)
len(d_edge_col_3)

496

### Fourth Disease Vector | DOID:0060089 - Endocrine Organ benign neoplasm

In [287]:
# -----------------------------
num_doid_4 = '0060089'
# -----------------------------

# define specific DOID 
dismod_4 = FM_disease.loc[:,'DOID:'+num_doid_4]
dismod_4.index = FM_disease.index
dismod_id_4 = 'DOID'+num_doid_4

dismod_genes_4 = [] 
for i,v in enumerate(dismod_4.values):
    if v == 1:
        dismod_genes_4.append(dismod_4.index[i])
    else:
        pass
    
l_genes_dismod_4 = [str(i) for i in dismod_genes_4]
print('Number of Genes associated to DOID', num_doid_4,':',len(l_genes_dismod_4))
                                                               
overlap_4 = []
for i in dismod_genes_4:
    if str(i) in nf_all or int(i) in nf_all:
    #if str(i) in dismod_genes or int(i) in dismod_genes:
        overlap_4.append(str(i))
#print('Number of Genes overlap of NF1 neighbors and Disease Module:', (len(overlap_4)))
      
l_genes_dismod_4_with = l_genes_dismod_4 + overlap_4 # + nf_neighbors[40:]
#print('Number of Genes associated to DOID', num_doid_4,':',len(l_genes_dismod_4_with))

# create a dataframe (sorted by DF global) and fill 1 when disease-assoc. gene
DF_dismod_4 = pd.DataFrame(dismod_4.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_4.index:
    if i in l_genes_dismod_4: #_with:
        DF_dismod_4.loc[i] = 1

Number of Genes associated to DOID 0060089 : 122


#### SPECIFIC NODE + EDGE COLORS

In [288]:
# color nodes
node_col_4 = '#992121' # '#A60C0C' # (darkred)

# color edges    
d_edge_col_4 = color_edges_from_nodelist_specific(G, l_genes_dismod_4_with, 
                                                  node_col_4)
len(d_edge_col_4)

118

## VECTOR : NF1 neighbors

In [291]:
overlap_merged = overlap_1 + overlap_2 + overlap_3 + overlap_4 

nf_rest = []
for i in nf_neighbors: 
    if i in overlap_merged:
        pass
    else:
        nf_rest.append(i)
        
DF_dismod_neighbors = pd.DataFrame(dismod.reindex(DF_structural.index ,fill_value=0))
for i in DF_dismod_neighbors.index:
    if i in nf_rest:
        DF_dismod.loc[i] = 1
    
DF_dismod_neighbors.columns = ['NF neighbours']
#print('Number of neighbor genes rest:', len(nf_rest))

##########################

# colors for NF1 neighbors

##########################

d_edge_col_neigh = color_edges_from_node(G, nf_goi, node_col_dismod)
print('Links of NF1 gene:',len(d_edge_col_neigh))

Links of NF1 gene: 49


# TO FIX: edges in between nf1neighbor genes and disease module genes should be highlighted 

In [303]:
import itertools as it
all_combinations = list(it.product(dismod_all, nf_neighbors))

In [305]:
matched = []
for link in all_combinations:
    if link in G.edges():
        matched.append(link)

In [310]:
len(matched)

1160

In [295]:
# Get Links between NF1 neighbor genes and Disease module genes

dismod_all = set(l_genes_dismod_1 + l_genes_dismod_2 + l_genes_dismod_3 + l_genes_dismod_4)
print('Number of all genes in disease modules:',len(dismod_all))

# from G.edges get all links that have NF1 neighbors 
neigh_links = []
for edge in G.edges():
    if edge[0] in nf_neighbors:
        neigh_links.append(edge[1])
    elif edge[1] in nf_neighbors:
        neigh_links.append(edge[0])
print('Number of links with nf neighbor genes:',len(set(neigh_links)))

Number of all genes in disease modules: 612
Number of links with nf neighbor genes: 7055


In [296]:
set_neigh_links = set(neigh_links)

l_linkergenes = []
for node in set_neigh_links:
    if node in dismod_all:
        l_linkergenes.append(node)
    else:
        pass

print('Number of linkergenes (nf1neighbor - disease module):', len(l_linkergenes))

Number of linkergenes (nf1neighbor - disease module): 380


In [297]:
# color nodes
node_col_neighdismodlinks = '#E800FF'

# color edges    
d_edge_col_neighdismodlinks = color_edges_from_nodelist_specific(G, l_linkergenes, node_col_neighdismodlinks)
len(d_edge_col_neighdismodlinks)

2431

# ALL DISEASE VECTORS multiplied with factor

In [252]:
# --------------------------------------
#
scalar_val_dismod = 2 #4 #2
#
# --------------------------------------
enhance_factor_dismod = round((1-l_max_visprob)*scalar_val_dismod,10)
print('ENHANCING FACTOR Disease Module: ',enhance_factor_dismod)


# --------------------------------------
#
scalar_val_discontext = 2 #4 #2
#
# -------------------------------------- 
enhance_factor_discontext = round((1-l_max_visprob)*scalar_val_discontext,10)
print('ENHANCING FACTOR Disease Context : ', enhance_factor_discontext)

ENHANCING FACTOR Disease Module:  0.187514
ENHANCING FACTOR Disease Context :  0.187514


In [253]:
# multiply with enhancing factor 

# Vector of NF1 Neurofibromatosis
DF_dismod_scal = DF_dismod * enhance_factor_dismod

# Vector of NF1 neighbor genes 
DF_dismod_neighbors_scal = DF_dismod_neighbors * enhance_factor_dismod #discontext 

DF_dismod_1_scal = DF_dismod_1 * enhance_factor_discontext
DF_dismod_2_scal = DF_dismod_2 * enhance_factor_discontext
DF_dismod_3_scal = DF_dismod_3 * enhance_factor_discontext
DF_dismod_4_scal = DF_dismod_4 * enhance_factor_discontext

# merge into one dataframe 
DF_merge_discontext = pd.concat([
                                DF_structural, # original global matrix 
                                    
                                DF_dismod_scal,  # Disease Module Vector 
                                DF_dismod_neighbors_scal, # NF1 Neighbor Genes Vector 
                        
                                DF_dismod_1_scal, # Disease Context Vectors 1-4
                                DF_dismod_2_scal,
                                DF_dismod_3_scal,
                                DF_dismod_4_scal,
    
                                ],
                                axis=1
                                )

DF_merge_discontext

,66008,8473,2561,3759,22906,4928,1994,8481,81610,51361,...,10861,51471,221044,29965,Neurofib 8712,NF neighbours,DOID:3165,DOID:10534,DOID:3620,DOID:0060089
66008,0.900242,0.000687,0.012865,0.006433,0.005347,0.000460,0.000051,0.000345,0.003464,0.002905,...,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
8473,0.002519,0.900245,0.000036,0.000018,0.005316,0.000005,0.000002,0.000003,0.000012,0.000008,...,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2561,0.002502,0.000002,0.900367,0.000018,0.000015,0.000002,0.000000,0.000001,0.000010,0.000008,...,0.000001,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3759,0.002502,0.000002,0.000036,0.900389,0.000015,0.000002,0.000000,0.000001,0.000010,0.000008,...,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
22906,0.002525,0.000685,0.000036,0.000018,0.900120,0.000459,0.000000,0.000001,0.000010,0.000009,...,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000070,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
10861,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.900070,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
51471,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.900052,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
221044,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.90003,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### 2D PORTRAIT POSITIONING 

In [254]:
n_n = 8 #6 #20
spr = 1.0
md = 0.0 #.4
metr = 'cosine'

In [255]:
%%time 
posG_umap2D = layout_portrait_umap(G,DF_merge_discontext, 2, 
                                   n_neighbors = n_n, 
                                   spread = spr, 
                                   min_dist = md, 
                                   metric = metr) 
posG = posG_umap2D

CPU times: user 1min 17s, sys: 12.9 s, total: 1min 30s
Wall time: 49.2 s


### VISUAL SETTINGS + PLOT PREPARATION

#### GENERAL NODE + EDGE COLORS 

In [256]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 2

scale_factor = 0.55
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

width_edges = 0.45
opacity_edges = 0.15

#### MERGE NODE COLORS > 4 diseases = 4 different colors 

In [268]:
# merge all node colors (and sort like G.nodes)

col_param = list(df_centralities['closeness'].values)
d_node_colors = dict(zip(list(G.nodes()), col_param))

c1='#E6E6E6' 
c2='#9B9B9B' 
n=len(set(col_param))

colors_for_palette = []
for x in range(n+1):
    colors_for_palette.append(colorFader(c1,c2,x/n))    

customPalette = sns.set_palette(sns.color_palette(colors_for_palette))
d_colors = color_nodes_from_dict(G, d_node_colors, palette = customPalette)

d_col_all = {}
for k,v in d_colors.items():
    
    # color for all NF1 associated genes (i.e. neighbors, NF1 disease module) 
    if k in l_genes_dismod or k in nf_neighbors:
        d_col_all[k] = node_col_dismod
        
    # disease module colors 
    elif k in l_genes_dismod_1:
        d_col_all[k] = node_col_1
        
    elif k in l_genes_dismod_2:
        d_col_all[k] = node_col_2
        
    elif k in l_genes_dismod_3:
        d_col_all[k] = node_col_3
    
    elif k in l_genes_dismod_4:
        d_col_all[k] = node_col_4
    
    else:
        d_col_all[k]=v
        
colors = list(d_col_all.values())
l_genes_all = l_genes_dismod + l_genes_dismod_1 + l_genes_dismod_2 + l_genes_dismod_3 + l_genes_dismod_4 + nf_neighbors 

### PLOT NETWORK PORTRAIT - plotly 

In [269]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_all:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())


umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth*0.25, 0.8)
umap_nodes_foreground_ = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, 5,
                                           None, 0.15)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) 
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 2, 
                                           None,0.5)

# DOID 8712 - NF1 edges
umap_edges_dismod = get_trace_edges_specific2D(d_edge_col_dismod,
                                               #posG, linew=0.25, opac=0.25)
                                               posG, linew=0.65, opac=0.55)

                                               
# NF1 gene links to other genes (direct neighbors) 
umap_edges_dismod_neigh = get_trace_edges_specific2D(d_edge_col_neigh, 
                                                     #posG, linew=0.5, opac=0.25)
                                                     posG, linew=0.45, opac=0.55)

# NF1neighbor-genes to Disease-module-genes 
umap_edges_dismod_neighdismodlinks = get_trace_edges_specific2D(d_edge_col_neighdismodlinks, 
                                                     #posG, linew=0.5, opac=0.25)
                                                     posG, linew=0.25, opac=0.25)

umap_edges_1 = get_trace_edges_specific2D(d_edge_col_1, 
                                          posG, linew=width_edges, opac=opacity_edges)
                                          #posG, linew=0.45, opac=0.55)

umap_edges_2 = get_trace_edges_specific2D(d_edge_col_2, 
                                          posG, linew=width_edges,opac=opacity_edges)
                                          #posG, linew=0.45, opac=0.55)

umap_edges_3 = get_trace_edges_specific2D(d_edge_col_3,
                                          posG, linew=width_edges, opac=opacity_edges)
                                          #posG, linew=0.45, opac=0.55)

umap_edges_4 = get_trace_edges_specific2D(d_edge_col_4, 
                                          posG, linew=width_edges, opac=opacity_edges)
                                          #posG, linew=0.45, opac=0.55)

data = [
        umap_nodes_background_, 
        umap_nodes_background, 
    
        #umap_edges_1, 
        #umap_edges_2,
        #umap_edges_3,
        #umap_edges_4,
    
        #umap_edges_dismod,
        #umap_edges_dismod_neigh,
    
        umap_edges_dismod_neighdismodlinks,
    
        umap_nodes_foreground_,
        umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
#fname = '2Dportrait_NF8712_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enhDismod'+str(scalar_val_dismod)+'_enhContext'+str(scalar_val_discontext)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism
fname = ' test'

plot_2D(data,path,fname)

'output_plots/2Dlayouts/ test.html'

# EXPORT POSITIONS >  VRNETZER Format 

In [259]:
path = 'output_plots/VRlayouts/' 
layout_nmspce = '2Dportrait_NF1'
cols = colors
export_to_csv2D(path, layout_nmspce, posG, cols)

# FIGURE - NF1 + 8712 + NEIGHBORS OF NF1
highlighting Neurofibromatosis + neighbors of gene NF1

In [175]:
# merge all node colors (and sort like G.nodes)

col_param = list(df_centralities['closeness'].values)
d_node_colors = dict(zip(list(G.nodes()), col_param))

c1='#E6E6E6' 
c2='#9B9B9B' 
n=len(set(col_param))

colors_for_palette = []
for x in range(n+1):
    colors_for_palette.append(colorFader(c1,c2,x/n))    

customPalette = sns.set_palette(sns.color_palette(colors_for_palette))
d_colors = color_nodes_from_dict(G, d_node_colors, palette = customPalette)

d_col_all = {}
for k,v in d_colors.items():
    
    if k in l_genes_dismod:
        d_col_all[k] = node_col_dismod

    elif k in nf_neighbors: 
        d_col_all[k] = node_col_dismod
    
    else:
        d_col_all[k]=v
        
colors = list(d_col_all.values())

In [107]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_dismod:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())


umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth*0.25, 0.8)
umap_nodes_foreground_ = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, 5,
                                           None, 0.2)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) #node_linewidth*0.05, 0.5)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 2, 
                                            None,0.5)

umap_edges_dismod = get_trace_edges_specific2D(d_edge_col_dismod, posG, linew=width_edges, 
                                               opac=0.5)# opacity_edges)
umap_edges_dismod_neigh = get_trace_edges_specific2D(d_edge_col_neigh, posG, linew=width_edges, 
                                                     opac=0.5) #opacity_edges)

data = [
        umap_edges_dismod,
        umap_edges_dismod_neigh,
        umap_nodes_background_, 
        umap_nodes_background, 
        umap_nodes_foreground_,
        umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname_NF1links = '2Dportrait_NF8712_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enhDismod'+str(scalar_val_dismod)+'_enhContext'+str(scalar_val_discontext)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism+'_NF1links'

plot_2D(data,path,fname_NF1links)

'output_plots/2Dlayouts/2Dportrait_NF8712_DOIDs3165_10534_3620_0060089_enhDismod2_enhContext2_nn10_spr1.0_md0.0_cosine_human_NF1links.html'

# FIGURE - ONLY NF1 + 8712
highlighting Neurofibromatosis

In [109]:
col_param = list(df_centralities['closeness'].values)
d_node_colors = dict(zip(list(G.nodes()), col_param))

c1='#E6E6E6' 
c2='#9B9B9B' 
n=len(set(col_param))

colors_for_palette = []
for x in range(n+1):
    colors_for_palette.append(colorFader(c1,c2,x/n))    

customPalette = sns.set_palette(sns.color_palette(colors_for_palette))
d_colors = color_nodes_from_dict(G, d_node_colors, palette = customPalette)

d_col_all = {}
for k,v in d_colors.items():
    if k in l_genes_dismod:
        d_col_all[k] = node_col_dismod
    
    else:
        d_col_all[k]=v
        
colors = list(d_col_all.values())

In [111]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes_dismod:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())


umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth*0.25, 0.8)
umap_nodes_foreground_ = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, 5,
                                           None, 0.2)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           None,0.5) #node_linewidth*0.05, 0.5)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 2, 
                                            None,0.5)

umap_edges_dismod = get_trace_edges_specific2D(d_edge_col_dismod, posG, linew=0.5,#width_edges, 
                                               opac=0.9) #opacity_edges)

data = [
        #umap_edges_dismod,
        umap_nodes_background_, 
        umap_nodes_background, 
        umap_nodes_foreground_,
        umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname_NF1module = '2Dportrait_NF8712_'+'DOIDs'+num_doid_1+'_'+num_doid_2+'_'+num_doid_3+'_'+num_doid_4+'_enhDismod'+str(scalar_val_dismod)+'_enhContext'+str(scalar_val_discontext)+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism+'_NF1module_nolinks'

plot_2D(data,path,fname_NF1module)

'output_plots/2Dlayouts/2Dportrait_NF8712_DOIDs3165_10534_3620_0060089_enhDismod2_enhContext2_nn10_spr1.0_md0.0_cosine_human_NF1module_nolinks.html'

# VERY MUCH WORK IN PROGRESS 

# 4. | STRUC + FUNC (complete disease matrix)

In [ ]:
FM_disease = pd.read_csv('input/Features_Disease_Dataframe_'+organism+'.csv', index_col=0)

In [ ]:
# --------------------------------------
#
scalar_val = 4
#
# --------------------------------------

enhance_factor = round((1-l_max_visprob)*scalar_val,10) # for emphasizing functional features > should be >= max of structural matrix values  # if significantly higher than max values in structural matrix > causes isolation of nodes with functional features enhanced
print('ENHANCING FACTOR functional features: ',enhance_factor)

n_n = 14
spr = 1
md = 0.0
metr = 'cosine'

In [ ]:
# create an empty matrix for zeros of rest genes (not associated to any disease)
rest = []
for i in G.nodes():
    if str(i) in FM_disease.index or int(i) in FM_disease.index:
        pass
    else: 
        rest.append(str(i))
        
empty = np.zeros(shape=(len(rest),len(FM_disease.columns)))
empty_stacked = np.vstack(empty)
df_empty = pd.DataFrame(empty_stacked)
df_empty.index = rest
df_empty.columns = FM_disease.columns

In [ ]:
# combine disease vector with rest of genes in graph
df_diseases_complete = pd.concat([FM_disease, df_empty])
df_diseases_complete

In [ ]:
# reorder according to G.nodes
DF_diseases_reorder = df_diseases_complete.reindex(DF_structural.index)

# fill nan positions (due to reordering) with 1 
DF_diseases_new = DF_diseases_reorder.replace(np.nan,1)

# multiply with enhancing factor
DF_diseases_scal = DF_diseases_new*enhance_factor 

DF_merge_diseases = pd.concat([DF_structural.T, 
                             DF_diseases_scal],axis=1)

DF_merge_diseases

### 2D PORTRAIT

In [ ]:
n_n = 12
spr = 1
md = 0.0
metr = 'cosine'

In [ ]:
%%time 

posG_umap2D = layout_portrait_umap(G,DF_merge_diseases, 2,
                                   n_neighbors = n_n,
                                   spread = spr,
                                   min_dist = md,
                                   metric = metr) 
posG = posG_umap2D

## VISUAL SETTINGS + PLOT PREPARATION

#### NODES - GENERAL

In [ ]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

scale_factor = 0.5
size_plotly = list(draw_node_degree(G, scale_factor/100).values())

In [ ]:
color_method = 'NF1'
l_genes = enhance_genes

node_col = '#00E3DF' #'#FF8E00' 
undefined_col = '#DEDEDE'
rest_col_nodes = '#DEDEDE'

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.5

## PLOT NETWORK PORTRAIT - plotly 

In [ ]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

d_size_plotly = draw_node_degree(G, scale_factor/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

In [ ]:
umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output_plots/2Dlayouts/'
schema = 'light' #'dark' 
fname = '2Dportrait_NetlayoutDisease_DiseaseFeatureMatrix_enhance'+str(enhance_factor)+'_'+color_method+'_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+str(metr)+'_'+organism

plot_2D(data,path,fname)